In [1]:
import pandas as pd
import numpy as np
from io import StringIO
import csv
import re
import fasttext


from sklearn.metrics import recall_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

import nltk
from nltk.tokenize import word_tokenize

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

In [2]:
stop_words = nltk.corpus.stopwords.words("russian")

In [3]:
def text_prepear2(text):

    text = re.sub(r'\S+#\S+;',' ', str(text))

    text = re.sub(r'#\S+',' ', text)

    text = re.sub(r'\S+#',' ', text)

    text = re.sub(r'_+',' ', text)

    text = re.sub(r'\W', ' ', text)

    text = re.sub(r'\s+[a-zA-Zа-яА-ЯёЁ]\s+', ' ', text)

    text = re.sub(r'\^[a-zA-Zа-яА-ЯёЁ]\s+', ' ', text)

    text = re.sub(r'\s+', ' ', text)

    text = text.lower()
    
    tokens = text.split()
    
    tokens = [word for word in tokens if word not in stop_words]
    
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

In [4]:
df1 = pd.read_csv('text_nodocs_actual_2355.csv')
df1['text_prep'] = df1['0'].apply(text_prepear2)
df1['target'] = np.ones((len(df1)))

In [7]:
df1.head(10)

,0,text_prep,target
0,"Документы с проблемами, в наличии только ГТД н...",документы проблемами наличии гтд кузов двигате...,1.0
1,"Продаю коня (Учет Армения) объёмом 3,7 (кабрио...",продаю коня учет армения объёмом 3 7 кабриолет,1.0
2,Состояние хорошее. Только наличный расчет. Ори...,состояние хорошее наличный расчет оригинальный...,1.0
3,Продаю заз 968 м с двигаиелем 2108 инжектор. Н...,продаю заз 968 двигаиелем 2108 инжектор ходу п...,1.0
4,3 месяца как пришла с Японии. Была в ИДЕАЛЬНОМ...,3 месяца пришла японии идеальном состоянии кор...,1.0
5,"Турбо форь, ej205 безфазный, люк, сигналка, на...",турбо форь ej205 безфазный люк сигналка ходу ц...,1.0
6,Сделка проходит под полным контролем сотрудник...,сделка проходит полным контролем сотрудников б...,1.0
7,ЦЕНА 400000 ТЫС!!!!!!!!!!!! торг уместен\n\nПр...,цена 400000 тыс торг уместен продам отличный с...,1.0
8,автомобиль имеет ограничения на регистрационны...,автомобиль имеет ограничения регистрационные д...,1.0
9,машина в хорошем состоянии без дырок коробка р...,машина хорошем состоянии дырок коробка работае...,1.0


In [8]:
df2 = pd.read_csv('text_nodocs_arh_5000.csv')
df2['text_prep'] = df2['0'].apply(text_prepear2)
df2['target'] = np.ones((len(df2)))

In [9]:
df2.head(10)

,0,text_prep,target
0,"продаю на разбор, не честично а вся. без номе...",продаю разбор честично вся номеров птс мошина ...,1.0
1,В ОТЛИЧНОМ СОСТОЯНИИ!!! ВЛОЖЕНО МНОГО ДЕНЕГ И ...,отличном состоянии вложено денег сил мосты воя...,1.0
2,"Машина в идеальном состоянии, пригнал пол года...",машина идеальном состоянии пригнал пол года на...,1.0
3,"автомобиль на ходу, действует запрет регистрац...",автомобиль ходу действует запрет регистрационн...,1.0
4,Сделка проходит под полным контролем сотрудник...,сделка проходит полным контролем сотрудников б...,1.0
5,"Продаю авто срочно !!! По мотору , ходовой и к...",продаю авто срочно мотору ходовой коробке наре...,1.0
6,Авто на армянских номерах . На мне на учете 2 ...,авто армянских номерах учете 2 года мелкие кос...,1.0
7,Сделка проходит под полным контролем сотрудник...,сделка проходит полным контролем сотрудников б...,1.0
8,Сделка проходит под полным контролем сотрудник...,сделка проходит полным контролем сотрудников б...,1.0
9,Сделка проходит под полным контролем сотрудник...,сделка проходит полным контролем сотрудников б...,1.0


In [10]:
df3 = pd.read_csv('text_docsok_37619.csv')
df3['text_prep'] = df3['0'].apply(text_prepear2)
df3['target'] = np.zeros((len(df3)))

In [11]:
df3.head(10)

,0,text_prep,target
0,&#x26a1; 30 000 СКИДКА В КРЕДИТ\n&#x26a1; 20 0...,30 000 скидка кредит 20 000 скидка обмене феде...,0.0
1,"Собственник. По технической части исправен, в ...",собственник технической части исправен дтп фак...,0.0
2,"Кондиционер,Сигнализация,MP3,Легкосплавные дис...",кондиционер сигнализация mp3 легкосплавные дис...,0.0
3,В продаже надежный Mazda CX -7. Двигатель 2.3 ...,продаже надежный mazda cx 7 двигатель 2 3 4 вл...,0.0
4,Данный автомобиль прошел комплексную диагности...,данный автомобиль прошел комплексную диагности...,0.0
5,Данный автомобиль прошел комплексную диагности...,данный автомобиль прошел комплексную диагности...,0.0
6,Модель: Jetta\n\nКомплектация: 1.6 AT (110 л.с...,модель jetta комплектация 1 6 at 110 кондицион...,0.0
7,Модель: Х-рей\n\nКомплектация: 1.8 AMT (122 л....,модель рей комплектация 1 8 amt 122 кондиционе...,0.0
8,Модель: Solaris\n\nКомплектация: 1.6 MT (123 л...,модель solaris комплектация 1 6 mt 123 кондици...,0.0
9,Модель: Octavia\n\nКомплектация: 1.6 MT (110 л...,модель octavia комплектация 1 6 mt 110 кондици...,0.0


In [12]:
df = pd.concat([df1, df2, df3], ignore_index=True)
df.drop('0', axis=1, inplace=True)
df['text_prep'] = df['text_prep'].apply(lambda x: 'for drop' if len(x)<5 else x)
df.drop(df[df['text_prep'] == 'for drop'].index, inplace=True)
df.index = range(len(df))

In [14]:
df.head(10)

,text_prep,target
0,документы проблемами наличии гтд кузов двигате...,1.0
1,продаю коня учет армения объёмом 3 7 кабриолет,1.0
2,состояние хорошее наличный расчет оригинальный...,1.0
3,продаю заз 968 двигаиелем 2108 инжектор ходу п...,1.0
4,3 месяца пришла японии идеальном состоянии кор...,1.0
5,турбо форь ej205 безфазный люк сигналка ходу ц...,1.0
6,сделка проходит полным контролем сотрудников б...,1.0
7,цена 400000 тыс торг уместен продам отличный с...,1.0
8,автомобиль имеет ограничения регистрационные д...,1.0
9,машина хорошем состоянии дырок коробка работае...,1.0


In [15]:
model = fasttext.load_model("fil2.bin")

In [16]:
x_mvec = []
for i in range(len(df)):
    x_mvec.append(np.mean([model.get_word_vector(x) for x in df['text_prep'][i].split()], axis=0))

In [17]:
dfm = pd.DataFrame(x_mvec)

In [18]:
dfm['target'] = df['target']

In [19]:
from sklearn.model_selection import train_test_split
X = dfm.drop(['target'], axis=1)
y = dfm['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [21]:
skfold = StratifiedKFold(n_splits=3, shuffle=True)

In [22]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(class_weight='balanced', max_iter=1000)

scores = cross_validate(logreg, X, y, cv=skfold, 
                        scoring=('accuracy', 'roc_auc', 'recall', 'precision', 'f1'))
df_sc = pd.DataFrame(list(scores.values())[2:], 
             index=['accuracy','roc_auc','recall', 'precision', 'f1'], 
             columns=['fold 1', 'fold 2', 'fold 3'])
df_sc['mean'] = list(df_sc.T.mean())
df_sc = round(df_sc, 3) 
df_sc

,fold 1,fold 2,fold 3,mean
accuracy,0.862,0.862,0.859,0.861
roc_auc,0.939,0.939,0.939,0.939
recall,0.873,0.875,0.882,0.877
precision,0.549,0.549,0.542,0.547
f1,0.674,0.675,0.672,0.673


In [23]:
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier()

scores = cross_validate(kn, X, y, cv=skfold, 
                        scoring=('accuracy', 'roc_auc', 'recall', 'precision', 'f1'))
df_sc = pd.DataFrame(list(scores.values())[2:], 
             index=['accuracy','roc_auc','recall', 'precision', 'f1'], 
             columns=['fold 1', 'fold 2', 'fold 3'])
df_sc['mean'] = list(df_sc.T.mean())
df_sc = round(df_sc, 3) 
df_sc

,fold 1,fold 2,fold 3,mean
accuracy,0.906,0.906,0.904,0.905
roc_auc,0.893,0.887,0.893,0.891
recall,0.581,0.582,0.575,0.579
precision,0.789,0.788,0.782,0.786
f1,0.669,0.669,0.663,0.667


In [24]:
from sklearn.svm import SVC
svc = SVC(class_weight='balanced')

scores = cross_validate(svc, X, y, cv=skfold, 
                        scoring=('accuracy', 'roc_auc', 'recall', 'precision', 'f1'))
df_sc = pd.DataFrame(list(scores.values())[2:], 
             index=['accuracy','roc_auc','recall', 'precision', 'f1'], 
             columns=['fold 1', 'fold 2', 'fold 3'])
df_sc['mean'] = list(df_sc.T.mean())
df_sc = round(df_sc, 3) 
df_sc

,fold 1,fold 2,fold 3,mean
accuracy,0.872,0.875,0.876,0.874
roc_auc,0.948,0.947,0.949,0.948
recall,0.891,0.877,0.884,0.884
precision,0.569,0.577,0.579,0.575
f1,0.695,0.696,0.700,0.697


In [25]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(class_weight='balanced')

scores = cross_validate(lgbm, X, y, cv=skfold, 
                        scoring=('accuracy', 'roc_auc', 'recall', 'precision', 'f1'))
df_sc = pd.DataFrame(list(scores.values())[2:], 
             index=['accuracy','roc_auc','recall', 'precision', 'f1'], 
             columns=['fold 1', 'fold 2', 'fold 3'])
df_sc['mean'] = list(df_sc.T.mean())
df_sc = round(df_sc, 3) 
df_sc

,fold 1,fold 2,fold 3,mean
accuracy,0.889,0.887,0.881,0.886
roc_auc,0.949,0.949,0.946,0.948
recall,0.833,0.829,0.840,0.834
precision,0.619,0.615,0.597,0.610
f1,0.710,0.706,0.698,0.705


In [ ]:
# #######

In [26]:
%%time
model = SVC()
param = dict(C=np.linspace(0.001, 2, 7), 
             kernel=['linear', 'rbf'], 
             random_state=[4210], 
             class_weight=['balanced'])
clf = GridSearchCV(model, param, cv=StratifiedKFold(n_splits=3, shuffle=True), scoring='roc_auc')
clf.fit(X, y)
print('roc_auc: {:.4}, with best: {}'.format(clf.best_score_, clf.best_params_))

roc_auc: 0.9483, with best: {'C': 2.0, 'class_weight': 'balanced', 'kernel': 'rbf', 'random_state': 4210}
Wall time: 39min 26s


In [27]:
from sklearn.svm import SVC
svc = SVC(class_weight='balanced', C=2)

scores = cross_validate(svc, X, y, cv=skfold, 
                        scoring=('accuracy', 'roc_auc', 'recall', 'precision', 'f1'))
df_sc = pd.DataFrame(list(scores.values())[2:], 
             index=['accuracy','roc_auc','recall', 'precision', 'f1'], 
             columns=['fold 1', 'fold 2', 'fold 3'])
df_sc['mean'] = list(df_sc.T.mean())
df_sc = round(df_sc, 3) 
df_sc

,fold 1,fold 2,fold 3,mean
accuracy,0.881,0.876,0.879,0.879
roc_auc,0.950,0.949,0.946,0.948
recall,0.880,0.880,0.875,0.878
precision,0.592,0.579,0.588,0.586
f1,0.708,0.698,0.703,0.703


In [48]:
%%time
model = LGBMClassifier()
param = dict(
    max_depth=[-1,3],
    min_child_weight=np.linspace(0.001, 1, 4),
    subsample=np.linspace(0.5, 1, 3),
    colsample_bytree=np.linspace(0.4, 1, 3),
    reg_alpha=np.linspace(0, 1, 3),
    random_state=[4210],
    class_weight=['balanced'])
clf = GridSearchCV(model, param, cv=StratifiedKFold(n_splits=3, shuffle=True), scoring='roc_auc')
clf.fit(X, y)
print('roc_auc: {:.4}, with best: {}'.format(clf.best_score_, clf.best_params_))

roc_auc: 0.9493, with best: {'class_weight': 'balanced', 'colsample_bytree': 0.7, 'max_depth': -1, 'min_child_weight': 0.334, 'random_state': 4210, 'reg_alpha': 0.5, 'subsample': 0.5}
Wall time: 8min 50s


In [57]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(class_weight='balanced',
                      min_child_weight= 0.334,
                      n_estimators=500,
                      colsample_bytree=0.7,
                      reg_alpha=0.5,
                      subsample=0.5)

scores = cross_validate(lgbm, X, y, cv=skfold, 
                        scoring=('accuracy', 'roc_auc', 'recall', 'precision', 'f1'))
df_sc = pd.DataFrame(list(scores.values())[2:], 
             index=['accuracy','roc_auc','recall', 'precision', 'f1'], 
             columns=['fold 1', 'fold 2', 'fold 3'])
df_sc['mean'] = list(df_sc.T.mean())
df_sc = round(df_sc, 3) 
df_sc

,fold 1,fold 2,fold 3,mean
accuracy,0.912,0.914,0.913,0.913
roc_auc,0.951,0.953,0.951,0.952
recall,0.761,0.752,0.744,0.752
precision,0.716,0.732,0.727,0.725
f1,0.738,0.742,0.736,0.738


In [ ]:
# #######

In [58]:
best_model = LGBMClassifier(class_weight='balanced',
                      min_child_weight= 0.334,
                      n_estimators=500,
                      colsample_bytree=0.7,
                      reg_alpha=0.5,
                      subsample=0.5)
best_model.fit(X, y)

LGBMClassifier(class_weight='balanced', colsample_bytree=0.7,
               min_child_weight=0.334, n_estimators=500, reg_alpha=0.5,
               subsample=0.5)

In [59]:
import joblib
joblib.dump(best_model, "model_docs.pkl") 

['model_docs.pkl']